<a href="https://colab.research.google.com/github/ale1211/data-analysis-freecodecamp/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries (NO instales nada)
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# load data
train_df = pd.read_csv(train_file_path, sep="\t", header=None, names=["label", "message"])
test_df = pd.read_csv(test_file_path, sep="\t", header=None, names=["label", "message"])

# Convert labels to 0 (ham) and 1 (spam)
train_df["label_num"] = train_df["label"].map({"ham": 0, "spam": 1})
test_df["label_num"] = test_df["label"].map({"ham": 0, "spam": 1})

# Prepare tokenizer
tokenizer = keras.preprocessing.text.Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(train_df["message"])

# Convert messages to sequences
x_train = tokenizer.texts_to_sequences(train_df["message"])
x_test = tokenizer.texts_to_sequences(test_df["message"])

# Pad sequences
max_len = 120
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=max_len, padding="post")
x_test = keras.preprocessing.sequence.pad_sequences(x_test, maxlen=max_len, padding="post")

# Labels
y_train = train_df["label_num"].values
y_test = test_df["label_num"].values
#celda 3

In [ ]:
#celda4
# Build the model
model = keras.Sequential([
    keras.layers.Embedding(10000, 16, input_length=max_len),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(16, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

# Train model
history = model.fit(
    x_train,
    y_train,
    epochs=10,
    validation_data=(x_test, y_test),
    verbose=1
)


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])


In [ ]:
#celda 5
def predict_message(pred_text):

    # Convert text → sequence → padded vector
    seq = tokenizer.texts_to_sequences([pred_text])
    padded = keras.preprocessing.sequence.pad_sequences(seq, maxlen=max_len, padding="post")

    # Predict likelihood of spam
    prob = model.predict(padded)[0][0]

    # FreeCodeCamp wants ham=0, spam=1
    label = "spam" if prob > 0.5 else "ham"

    return [float(prob), label]

pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
